# Evaluation of Generated Molecules
To evaluate the generated molecules, a DFT-based geometry optimization is performed using orca.
When training with implicit hydrogen, hydrogen atoms can be added using the software Avogadro.

![Validation of generated molecules](../fig/orca_validation.png)

## Creating ORCA Input Files
The following cell creates an orca input file for all molecules given as .xyz files in the directory dft_dir.

In [1]:
import os


def write_orca_input(xyz_path):
    """Write an orca input file for a molecule given as xyz file"""
    with open(xyz_path, 'r') as f:
        xyz_lines = f.readlines()

    output = "! PBE0 OPT def2-SVP \n"
    output += "*xyz 0 1\n"
    output += ''.join(xyz_lines[2:])
    output += "*\n"

    with open(xyz_path[:-3] + 'inp', 'w') as f:
        f.write(output)


dft_dir = 'sampled'
xyz_files = [os.path.join(dft_dir, f) for f in os.listdir(dft_dir) if f.endswith('.xyz')]
for xyz_file in xyz_files:
    write_orca_input(xyz_file)

## After the Optimization
After the optimization, we check if it has converged and, if it has, convert the results back to an xyz file.
For this, we need to put the log orca wrote to stdout as \*.log and the files \*_property.txt into the dft_dir.

In [6]:
def has_converged(log_file):
    with open(log_file, 'r') as f:
        log_content = ''.join(f.readlines())
    return 'HURRAY' in log_content


def prop_to_xyz(prop_file):
    """Read the last set of coordinates from the *_property.txt and return it in xyz format."""
    geometry = '!GEOMETRY!'
    with open(prop_file, 'r') as f:
        lines = f.readlines()
        geom_i = None
        for i in range(1,40):
            if geometry in lines[-i]:
                geom_i = -i
            
        atoms_line = lines[geom_i + 1]
        atoms_line = " ".join(atoms_line.split()) # remove duplicate whitespace
        num_atoms = atoms_line.split(' ')[3]
        
        coordinate_lines = lines[geom_i + 3:]
        coordinate_lines = [line.strip(' ') for line in coordinate_lines]
        coordinates = ''.join(coordinate_lines)
        xyz = num_atoms + '\n\n' + coordinates
        return xyz
    

log_files = [os.path.join(dft_dir, f) for f in os.listdir(dft_dir) if f.endswith('.log')]

for log in log_files:
    mol_name = os.path.splitext(os.path.basename(log))[0]
    if has_converged(log):
        prop_file = os.path.join(dft_dir, mol_name + '_property.txt')
        with open(os.path.join(dft_dir, mol_name+'_dft.xyz'), 'w') as f:
            f.write(prop_to_xyz(prop_file))

Finally, we can evaluate the deviation in atom positions between the generated and relaxed geometry.

In [19]:
import openbabel
import pybel
import numpy as np


def align_mols(ref, to_be_aligned):
    align = openbabel.OBAlign(False, False)
    align.SetRefMol(ref)
    align.SetTargetMol(to_be_aligned)
    align.Align()
    align.UpdateCoords(to_be_aligned)


def mse_mae(pybel_mol_1, pybel_mol_2):
    mse, mae, num_atoms = 0, 0, 0
    for atom_1, atom_2 in zip(pybel_mol_1.atoms, pybel_mol_2.atoms):
        x1, y1, z1 = atom_1.coords
        x2, y2, z2 = atom_2.coords
        squared_dist = (x1 - x2) ** 2 + (y1 - y2) ** 2 + (z1 - z2) ** 2
        mse += squared_dist
        mae += np.sqrt(squared_dist)
        num_atoms += 1

    return mse / num_atoms, mae / num_atoms


def eval_position_errors(xyz_1, xyz_2):
    py_mol_1 = pybel.readstring('xyz', xyz_1)
    py_mol_2 = pybel.readstring('xyz', xyz_2)
    
    py_mol_1.removeh()
    py_mol_2.removeh()
    align_mols(py_mol_1.OBMol, py_mol_2.OBMol)

    mse, mae = mse_mae(py_mol_1, py_mol_2)
    return mse, mae


dft_files = [os.path.join(dft_dir, f) for f in os.listdir(dft_dir) if f.endswith('dft.xyz')]
mses, maes = {}, {}

for dft in dft_files:
    mol_name = os.path.splitext(os.path.basename(dft))[0].split('_')[0]
    orig = os.path.join(dft_dir, mol_name + '.xyz')
    
    with open(dft, 'r') as f:
        xyz_dft = ''.join(f.readlines())
    with open(orig, 'r') as f:
        xyz_orig = ''.join(f.readlines())
    mse, mae = eval_position_errors(xyz_dft, xyz_orig)
    mses[mol_name] = mse
    maes[mol_name] = mae
    
    
maes_list = list(maes.values())
mses_list = list(mses.values())
mae_mean, mae_std = np.mean(maes_list), np.std(maes_list)
print('MAE: {:0.2f} +- {:0.2f}'.format(mae_mean, mae_std))
print('RMSE: {:0.2f}'.format(np.sqrt(np.mean(mses_list))))


MAE: 0.30 +- 0.12
RMSE: 0.36
